In [0]:
!pip install transformers

     |████████████████████████████████| 665kB 8.4MB/s 
     |████████████████████████████████| 890kB 23.8MB/s 
     |████████████████████████████████| 3.8MB 54.3MB/s 
     |████████████████████████████████| 1.1MB 50.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fcf9ac1ccec2b3555ac4801477c412c2dec54e46045f9d075ec57888ddb69cf8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [0]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
import pandas as pd
!gdown --id 12_VY9LDKd6WBKzNnE_oNsSrjMUR_BTrO
df = pd.read_csv('parsed-with_summaries_frequency.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=12_VY9LDKd6WBKzNnE_oNsSrjMUR_BTrO
To: /content/parsed-with_summaries_frequency.csv
18.2MB [00:00, 57.8MB/s]


,title,speaker,transcript,year,summary
0,Congressional Gold Medal Acceptance Address,Aung San Suu Kyi,This is one of the most moving days of my life...,2012,How privileged I am to be with you here today...
1,Memorial Remarks for Ronald Reagan,Prime Minister Brian Mulroney,"In the spring of 1987, President Reagan and I ...",2004,And here we were waiting for our wives. They ...
2,Address to the American Society of Newspaper E...,Dwight D. Eisenhower,"President Bryan, distinguished guests of this ...",1953,I am happy to be here for another reason. It ...
3,2004 Democratic National Convention Address,Al Gore,"Thank you, very much. Thank you. Thank you, ve...",2004,"Thank you, very much. Thank you. Thank you, v..."
4,Speech to the D.C. Federalist Society Lawyers ...,Edwin Meese III,A large part of American history has been the ...,1985,"Now, as we approach the bicentennial of the f..."


In [0]:
# test case
text = df.summary[0]
print('text:', text)

text:  How privileged I am to be with you here today. And these are the people we need to remember -- not just those in my country, but everywhere else in the world where freedom is yet a dream. To be a whole human being, we need both security and freedom. No doubt, it is not perfect. We have been united, we from many parts of the world, by our belief, by our confidence in democracy. Thank you.


In [0]:
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

original text preprocessed: 
 How privileged I am to be with you here today. And these are the people we need to remember -- not just those in my country, but everywhere else in the world where freedom is yet a dream. To be a whole human being, we need both security and freedom. No doubt, it is not perfect. We have been united, we from many parts of the world, by our belief, by our confidence in democracy. Thank you.


In [0]:
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

In [0]:
# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True )

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)



Summarized text: 
 to be a whole human being, we need both security and freedom. no doubt, it is not perfect. we have been united by our belief, by confidence in democracy. thank you.


In [0]:
def t5_summarization(i, text):

  tokenized_text = tokenizer.encode(text, return_tensors="pt", max_length = 512).to(device)
  # max sequence the model can handle is 512 tokens
  # tokenized_text = tokenized_text[:512]
  #summarize
  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  print (i, "\nOriginal text:", text, "\n\nSummarized text: \n", output, '--------------------------')
  return output

In [0]:
for i, text in enumerate(df.transcript):
  df.loc[df.index[i],'summary'] = t5_summarization(i, df['transcript'][i])

Streaming output truncated to the last 5000 lines.
In any event, the prospect that the courts can meaningfully resolve inter-branch disputes about the meaning of the Constitution is mostly a false promise. How is a court supposed to decide, for example, whether Congress’s power to collect information in pursuit of its legislative function overrides the President’s power to receive confidential advice in the pursuit of his executive function? Nothing in the Constitution provides a manageable standard for resolving such a question. And it is thus no surprise that the courts have produced amorphous, unpredictable balancing tests like the Court’s holding in Morrison v. Olson -- that Congress did not “disrupt the proper balance between the coordinate branches by preventing the Executive Branch from accomplishing its constitutionally assigned functions.”
Apart from...their overzealous role in inter-branch disputes, the courts have increasingly engaged directly in the usurpation of Presidenti